# Udacity Data Analyst Project 3:  Wrangling Open Street Data

_ by Jens Laufer _

_ jenslaufer@gmail.com _


## Introduction

In this project I am importing XML Data from OpenStreetMap, auditing and wrangling this data and export it to MongoDB. Afterwards I am doing some analysis on the imported data.

I selected the area of Ostfriesland in north eastern Germany, which is close to the north sea. Although I am not from there, I am interested in the area as I want to extend my Airbnb hosting business and the area looks promising to me, as it is touristic and property prices are low. I got the insights about the area from google search data and data from the german realestate website [Immobilienscout24](http://immobilienscout24.de), but this is not part of the assigment. I am especially interesting in all data related to lodging. 

![](img/map.png)

I moved the code from the case study from the Udacity Data Analyst nanodegree course to a module called osm.py, which I am using within the scope of this assignment.

In [1]:
# python imports
from collections import defaultdict
import pprint
import re
import codecs
import json
import os
import file_size_humanize as humanize
import os.path as path
import time
from pprint import pprint
import pymongo as mongo
# this is the import of the code from the case study
import osm
#import of extractor with code provided from udacity 
import extractor

In [2]:
"""
definition of constants which I am using in this notebook 
"""

DO_IMPORT = False

OSM_URL = "http://overpass-api.de/api/map?bbox=6.6309,53.4302,7.8291,53.8227"
OSM_FILE = "ostfriesland.osm" 
OSM_EXTRACT_FILE = "{0}_extract.osm".format(OSM_FILE[:OSM_FILE.find('.osm')])

OSM_EXPORT_FILE = "{0}.json".format(OSM_FILE)

MONGO_URL = 'mongodb://localhost:27017/'

## Download of the dataset

In [ ]:
"""
I stream the osm data with the overpass url for the box I want to use into a local file in case the file does not already exist
"""
from urllib2 import urlopen

if not path.exists(OSM_FILE):
    response = urlopen(OSM_URL)
    CHUNK = 16 * 1024
    with open(OSM_FILE, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)

## Auditing of the data

#### Auditing contact data

I auditing the contact data. Therefore I created a audit_contact_data in the osm module, which checks email address and URLs against a regular expression. For the phonenumber I am using a port of [Google's libphonenumber for python](https://github.com/daviddrysdale/python-phonenumbers) to test for phonenumber validity.

In [ ]:
invalid_phone, invalid_email, invalid_url = osm.audit_contact_data(OSM_FILE)

Error parsing 0800 283 50000: (0) Missing or invalid default region.


In [ ]:
invalid_phone

In [ ]:
invalid_email

In [ ]:
invalid_url

I need to fix this problematic data, before exporting the data to MongoDB.

## Export of OSM to JSON and Import to MongoDB

I am extracting the data I am interested to, fixing the problematic data entries and export it to JSON for the MongoDB import.

In [ ]:
if DO_IMPORT:
    start = time.time()
    osm.process(OSM_FILE,OSM_EXPORT_FILE);
    (time.time() - start)

We add an index on id to improve speed on upsert on the id field

In [ ]:
nodes = mongo.MongoClient(MONGO_URL).osm.nodes
nodes.create_index([("id", mongo.ASCENDING)]);


In [ ]:
if DO_IMPORT:
    start = time.time()
    os.system('mongoimport --quiet --upsertFields id --db osm --collection nodes --file {0}'.format(OSM_FILE+'.json'));
    (time.time() - start)

### Addition of some indexes

In [ ]:
nodes.create_index([("type", mongo.ASCENDING)])
nodes.create_index([("address.city", mongo.ASCENDING)])

nodes.create_index([("pos", mongo.GEO2D)]);

## Analysis of the data

#### Tags in OSM file

In [ ]:
osm.count_tags(OSM_FILE)

#### File sizes

In [ ]:
# File size of the full osm file
info = os.stat(OSM_FILE)
"Filesize of {0} {1}".format(OSM_FILE, humanize.humansize(info.st_size))

In [ ]:
# File size of the full osm file
info = os.stat(OSM_EXPORT_FILE)
"Filesize of {0} {1}".format(OSM_EXPORT_FILE, humanize.humansize(info.st_size))

#### Number of documents in database

In [ ]:
"{} Documents in MongoDB".format(nodes.find().count())

#### Example document in MongoDB

In [ ]:
pprint(nodes.find_one({ "contact.phone": { '$exists': 1 }, "contact.fax": { '$exists': 1 }, 
            "contact.website": { '$exists': 1 }, "address.street": { '$exists': 1 }, 
            "address.city": { '$exists': 1 } }))

#### Document types

In [ ]:
pprint(list(nodes.aggregate([
        {'$group':{'_id':'$type','count':{'$sum':1}}}
    ])))

#### Documents with contact data

In [ ]:
"{} Documents with contact in MongoDB".format(nodes.find({'contact':{'$exists':1}}).count())

#### Documents with address data

In [ ]:
"{} Documents with address in MongoDB".format(nodes.find({'address':{'$exists':1}}).count())

#### Test Fixing of problematic data

I am checking if my code for importing data fixed the problematic phone numbers and URLs.

In [ ]:
nodes.find({'contact.phone':{'$in':invalid_phone}}).count()

In [ ]:
nodes.find({'contact.phone':{'$in':['+49 800 28350000', '+49 162 2119066', '+49 173 2922190']}}).count()

In [ ]:
nodes.find({'contact.website':{'$in':invalid_url}}).count()

In [ ]:
nodes.find({'contact.website':{'$in':
['http://www.silvis-bungalow.de',
 'http://www.haus-thomas.de',
 'http://www.tuedelpott.de',
 'http://www.hotel-cafecaro.de',
 'http://www.hotel-westfalenhof.de']}}).count()

#### Analyis of lodging data

What type of touristic data is there?

In [ ]:
pprint(list(nodes.aggregate([
   {'$match': {'tourism':{'$exists':1}}},
   {'$group':{'_id':'$tourism'}}
        ])))

I am interested in the number of lodging facilities in the area:

In [ ]:
pprint(list(nodes.aggregate([
                {'$match':{'tourism':{'$in':['bed_and_breakfast','motel','apartment','hostel','guest_house','chalet','hotel']}}},
                {'$group':{'_id':'$address.city', 'count':{'$sum':1}}},
                {'$sort':{'count':-1}}
            ])))

It is interesting, that for 112 lodging facilities no city is in the data set:

In [ ]:
pprint(list(
    nodes.aggregate([
            { '$match':
              {'tourism':{'$exists':False},
               'name':{'$regex': '([Pp]ension)|([Ff]erienwohnung)|([Ff]e[wW]o)|([Aa]partment)|([Ch]alet)|([Jj]ugendherberge)'}}
            },
            {'$count':'id'}
                
            ])
    ))

## Additional problems in the data

  - Cross field consistecy Postcode ork for Street and City. 
  - Have address always a housenumber in case ther is a street
  - Handlingof P.O. boxes
  - E-Mail address still valid
  - Website still available
  - Street names
  - City names
  - Phone number: In case we don't have the country how to get international format?